In [4]:
search_for = 124
start_from = 0
threads = 10
thread = 9
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.003954648971557617
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 75
all cases: 621252555
type: [1, 1, 1, 1, 124] 124
cases of this type: 236421376
100000 25.11051652682198
200000 42.18104349187777
300000 44.915630176536226
400000 43.443968369585846
500000 45.04529554855536
600000 43.75481864670429
700000 42.96208847515238
800000 43.594929163643926
900000 42.61398040909988
1000000 15.358118254612341
1100000 44.318334211482444
1200000 42.60728920445195
1300000 42.32838649190114
1400000 42.73405781515168
1500000 44.62704048721695
1600000 42.862783135737594
1700000 42.787076037723196
1800000 43.77172020456874
1900000 37.27033757272916
2000000 21.597064957305157
2100000 42.27021893941963
2200000 42.18301930906005
2300000 41.44455089167461
2400000 43.13156071870244
2500000 42.39563194359137
2600000 41.29696564128694
2700000 40.952151557892066
2800000 40.95311847674557
2900000 17.30803371966587
3000000 40.2304053

29700000 43.216445323589845
29800000 42.49980298571487
29900000 42.460525383006285
30000000 44.53263934391299
30100000 44.57915389291182
30200000 45.54568009850854
30300000 27.37889270995638
30400000 39.82192075176266
30500000 46.74926645522352
30600000 44.45705678500091
30700000 44.29746820051235
30800000 20.457968765255902
30900000 45.25176563425138
31000000 45.56101450480734
31100000 43.379919997033014
31200000 43.573429984082786
31300000 40.262548114683206
31400000 39.55522933772483
31500000 17.16697383926659
31600000 40.872018169877194
31700000 39.405372388714554
31800000 42.0081347189695
31900000 41.81441359738445
32000000 44.79266451149244
32100000 43.61840541039294
32200000 24.57985111527508
32300000 42.64470145085344
32400000 43.799370021149564
32500000 46.57361498265221
32600000 45.34091875577027
32700000 27.44506106231281
32800000 38.77522710644105
32900000 45.20277982654265
33000000 44.6044240148448
33100000 43.62856895782038
33200000 41.78847531168771
33300000 40.327992796

59800000 38.07492313552224
59900000 37.1531193410309
60000000 38.20327034127909
60100000 15.004127442701693
60200000 38.50788156785351
60300000 37.6882211162645
60400000 37.900828454806444
60500000 37.64678135037097
60600000 20.550987588884464
60700000 40.187075394761294
60800000 41.42739547795828
60900000 41.71907758588159
61000000 43.407986304877184
61100000 41.21591897420332
61200000 40.25985360795583
61300000 40.01196599318851
61400000 39.553454004537336
61500000 32.69530288393712
61600000 24.720371230133722
61700000 35.740671818910485
61800000 38.64247437787234
61900000 35.54462260393425
62000000 17.8319222030514
62100000 34.385206790481426
62200000 37.714964477463184
62300000 35.332358613248324
62400000 37.056141146096756
62500000 21.071813167535307
62600000 38.61345846444446
62700000 39.7584747611587
62800000 40.18698196640767
62900000 40.90176134151633
63000000 41.56319901523545
63100000 41.032542910580865
63200000 41.59719374363006
63300000 39.2356321244215
63400000 39.3090325

89900000 36.65875462382608
90000000 36.025704830904196
90100000 36.035307169343504
90200000 35.3886379176554
90300000 34.92363518368499
90400000 16.241229265564122
90500000 32.69840407095809
90600000 16.06116680225098
90700000 32.25143688006828
90800000 27.274562749517155
90900000 26.123124261181335
91000000 35.72102734144479
91100000 34.428471849649206
91200000 35.87096988876018
91300000 36.14734382333888
91400000 38.0580154841039
91500000 38.57222738851341
91600000 37.92318646195422
91700000 37.393536973780265
91800000 35.16056256959238
91900000 33.35947909371437
92000000 34.35852604472749
92100000 33.76833165193975
92200000 33.66645568402152
92300000 16.118792127320297
92400000 33.51107093479976
92500000 17.055641267892277
92600000 30.323591123306848
92700000 23.43046709125318
92800000 27.780245423520864
92900000 34.36422388238274
93000000 34.24224207340614
93100000 33.494688084775994
93200000 34.034920603449045
93300000 36.34631106466165
93400000 37.74120809426864
93500000 39.29226

119200000 3.670210189610223
119300000 3.640124246191667
119400000 3.6628722476723494
119500000 3.6995999583414325
119600000 3.6888784059808053
119700000 3.6871189955948096
119800000 3.786296806230254
119900000 3.6708028683215943
120000000 3.6942444852509
120100000 3.7039991805578536
120200000 3.737822254174163
120300000 3.7219788360873762
120400000 3.6875800906959273
120500000 3.632171427304102
120600000 3.657021334581375
120700000 3.731062347794318
120800000 3.708205164240658
120900000 3.6704640373680992
121000000 3.617730614870174
121100000 3.7252497021155424
121200000 3.6300510616470874
121300000 3.6298906559211446
121400000 3.642145651650813
121500000 3.6885495542525644
121600000 3.6058299782936905
121700000 3.656254941789674
121800000 3.639527449473977
121900000 3.7090051056619258
122000000 3.7225758307889536
122100000 3.825975092144191
122200000 3.6355255905419357
122300000 3.6834059635732403
122400000 3.619482805658559
122500000 3.650927860993806
122600000 3.6579351298772096
122

...

492900000 4.199600336986035
493000000 4.371683723965277
493100000 4.906809320159764
493200000 4.570454641478389
493300000 4.224428533358756
493400000 1.5025256125512854
493500000 3.8324174023770983
493600000 3.39402455051802
493700000 3.182204572967211
493800000 3.8876670724865194
493900000 4.1801661666261785
494000000 4.241658034104059
494100000 5.023964994043152
494200000 4.251617950151169
494300000 2.723937808857878
494400000 2.711266084150215
494500000 3.6307828913471667
494600000 2.594068489536004
494700000 3.978770892863393
494800000 4.2239955130252405
494900000 4.333427978917436
495000000 4.262987517539263
495100000 4.614745637342897
495200000 4.378538447208196
495300000 1.3701577708905934
495400000 3.875063825503578
495500000 2.447780408058693
495600000 4.017846351974428
495700000 4.109788196199583
495800000 4.133741277680503
495900000 4.127502904106239
496000000 4.285368508241375
496100000 4.599683882551137
496200000 2.746704879027406
496300000 2.6327275188348325
496400000 2.8

521000000 3.4295922124623326
521100000 3.335720657996714
521200000 3.251348578377737
521300000 3.2213065194059345
521400000 3.3163590816860995
521500000 3.3385606025800936
521600000 3.2633177761972516
521700000 3.2327785468031167
521800000 3.2907994885451526
521900000 3.337292019427998
522000000 3.361225256618818
522100000 3.293952719873147
522200000 3.4431071862910785
522300000 3.3989001236996055
522400000 3.483444739099198
522500000 3.385318499596066
522600000 3.311341548027336
522700000 3.494008632694049
522800000 3.4478961481800146
522900000 3.274550184027553
523000000 3.3683642554591597
523100000 3.286601577093243
523200000 3.5043236383476954
523300000 3.352983532177809
523400000 3.4505582445960377
523500000 3.157451619428158
523600000 3.0742117043100397
523700000 3.168485743057791
523800000 3.2451897435662844
523900000 3.250778805507643
524000000 3.248603923901498
524100000 3.1402534939749143
524200000 3.351713913481484
524300000 3.104485593704681
524400000 3.8577458470803796
524

547900000 3.782542747163928
548000000 2.186196013955093
548100000 3.3996379546789925
548200000 3.9622480527588895
548300000 1.185682741957721
548400000 0.559424456773758
548500000 0.5582846050151454
548600000 0.5567792771042751
548700000 0.5588426587556898
548800000 0.5552384022426969
548900000 0.5453610775372418
549000000 0.5501348279687961
549100000 0.55679888589359
549200000 0.5524001043651502
549300000 0.5528726871725123
549400000 0.557279048946768
549500000 0.5508078290646043
549600000 0.5489039506957233
549700000 0.5673330952860547
549800000 0.5640054474354255
549900000 0.5521813226267298
550000000 0.5508942208624614
550100000 0.5582760321473115
550200000 0.5441197221591969
550300000 0.5411647991256251
550400000 0.5440220215207503
550500000 0.5379506720819772
550600000 0.5640212940429582
550700000 4.572659899384075
550800000 4.2555666283689435
550900000 1.5760659693541759
551000000 4.334606165029095
551100000 4.454423374574086
551200000 3.3866534393204946
551300000 2.765797695532

576200000 0.8012027975254059
576300000 1.4019915942715706
576400000 1.3526218202734224
576500000 0.7036399751608073
576600000 0.7818874503680467
576700000 1.0157286130152046
576800000 1.4126281214718917
576900000 0.9104367000973025
577000000 0.7855296386105782
577100000 0.44387342172088895
577200000 0.981379170443525
577300000 0.9397893706280525
577400000 0.7766239401397738
577500000 0.7869827824588715
577600000 0.6732938235264685
577700000 1.0897111453622048
577800000 0.9611367141918243
577900000 1.144168940011412
578000000 0.6069041387527585
578100000 0.6697020381629766
578200000 1.2395901124235291
578300000 0.9081125025105741
578400000 1.3444195713750124
578500000 0.7044444120026628
578600000 1.1677261493630442
578700000 1.4625753276406526
best so far: 0
type: [1, 2, 1, 31, 2] 124
cases of this type: 953312
578800000 0.4097548162951403
578900000 0.2645798849051462
579000000 0.2755059849277834
579100000 0.27571611342627145
579200000 0.2740481040541629
579300000 0.2790166987215579
579

599600000 0.1587099816608197
599700000 0.14206372717750074
599800000 0.21755716178031761
599900000 0.5331074635770652
600000000 0.6209855872010291
600100000 0.7640996472705569
600200000 0.4083534076744119
600300000 0.7578919737231533
600400000 0.33719890018347237
600500000 0.5151779331069721
600600000 0.44014002399859825
600700000 0.56684721785196
600800000 0.6001908551595642
600900000 0.45230988030782343
601000000 0.45146144009211997
601100000 0.5208315291729239
601200000 0.6016534870580137
601300000 0.5478527792787652
601400000 0.4302923476988674
601500000 0.7079200995575089
601600000 0.37045387727596363
601700000 0.4745900285633637
601800000 0.4251125031914413
601900000 0.33865206135351794
602000000 0.3831869720573822
602100000 0.3954895433823963
602200000 0.6486991272895501
602300000 0.4334837091580464
602400000 0.5409130840227505
602500000 0.6129887551355329
602600000 0.5120985983851254
602700000 0.6440202981829941
602800000 0.5041009780928857
602900000 0.6323482220447328
60300000